# RNNLM
Recurrent Neural Network Language Model
RNN による言語モデルです。
文章の集団を学習させることで、それっぽい文章を生成できます。

これが発展して Seq2Seq のデコーダー部分になっていきます。

In [1]:
# カレントディレクトリをリポジトリ直下にするおまじない
import os
while os.getcwd().split('/')[-1] != 'deep_dialog_tutorial': os.chdir('..')
print('current dir:', os.getcwd())

current dir: /home/harumitsu.nobuta/git/deep_dialog_tutorial


In [2]:
import tensorflow as tf
from tensorflow.python.layers import core as layers_core
import numpy as np
import os
import random
import collections
from tqdm import tqdm

# Create Model

In [3]:
hidden_dim = 1024
embedding_dim = 256
vocab_size = 1000

In [4]:
# 入出力部分
in_ph = tf.placeholder(tf.int32, shape=[None, None], name='in_ph')
out_ph = tf.placeholder(tf.int32, shape=[None, None], name='out_ph')
len_ph = tf.placeholder(tf.int32, shape=[None], name='len_ph')
gen_start_token_ph = tf.placeholder(tf.int32, shape=[], name='gen_start_token_ph')

In [5]:
def debug(ops):
    '''与えられた計算ノードの値を表示します。'''
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        result = sess.run(ops, {
            in_ph: [[30, 40, 50], [160, 170, 180]],
            out_ph:[[40, 50, 60], [170, 180, 190]],
            len_ph:[3, 3]
        })
        print('## {}\nshape: {}'.format(ops.name, ops.shape))
        print(result)

In [6]:
# embeddings - 文字の ID から分散表現のベクトルに変換します。
# データは [batch_size, sentence_len, embedding_dim] の形になります。
embeddings = tf.Variable(tf.random_normal([vocab_size, embedding_dim], stddev=1), name='embeddings', dtype=tf.float32)
in_embedded = tf.nn.embedding_lookup(embeddings, in_ph)
debug(in_embedded)

## embedding_lookup/Identity:0
shape: (?, ?, 256)
[[[-0.20361237 -0.11308474 -0.41880432 ...  2.7252874  -1.0681399
    1.3086659 ]
  [-0.3352617  -1.0744342   0.9656708  ...  1.1087787   1.8505251
    0.02086403]
  [-0.48931482  1.2667885   0.58199185 ...  0.02560114  0.500132
   -3.2564793 ]]

 [[-0.12822877 -0.22769526  1.352034   ...  1.9360523  -0.34742078
   -0.40487522]
  [-2.3179045   0.57485205 -0.754861   ... -1.3065025   1.2339923
    0.01515338]
  [ 0.6592995  -0.6290501  -0.36402264 ...  0.42282453  0.3705973
    0.5638999 ]]]


In [7]:
# RNN 部分
cell = tf.nn.rnn_cell.GRUCell(hidden_dim, kernel_initializer=tf.orthogonal_initializer)
rnn_out, final_state = tf.nn.dynamic_rnn(
    cell=cell,
    inputs=in_embedded,
    sequence_length=len_ph,
    dtype=tf.float32,
    scope='rnn',
)
# 隠れ層から全結合をかませて、各単語の生成確率っぽい値にする。
# （i番目のニューロンの出力が id: i の単語の生成確率っぽいものになる）
output_layer = layers_core.Dense(vocab_size, use_bias=False, name='output_layer')
onehot_logits = output_layer.apply(rnn_out)
debug(onehot_logits)
output_ids_op = tf.argmax(onehot_logits, -1)

## output_layer/Tensordot:0
shape: (?, ?, 1000)
[[[ 0.02734021  0.11152687 -0.12108772 ... -0.02212429 -0.09032315
    0.03886116]
  [ 0.15023798  0.12774038  0.01580057 ...  0.13109946 -0.19433439
    0.08362214]
  [ 0.11481191  0.05790965 -0.00183003 ...  0.05068074 -0.42197746
    0.042301  ]]

 [[ 0.17572726  0.10455676 -0.05352221 ...  0.10981397 -0.15552926
   -0.07315858]
  [ 0.07044679 -0.0157798   0.10679971 ...  0.0757293  -0.04247508
    0.08267353]
  [-0.11602538 -0.07365637 -0.05860609 ...  0.08297199 -0.10835387
    0.24690573]]]


In [8]:
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=out_ph,
    logits=onehot_logits,
)
loss_op = tf.reduce_mean(cross_entropy, name='loss')
debug(loss_op)

## loss:0
shape: ()
6.907025


In [9]:
# 生成時用の RNN
beam_width = 20
gen_max_len = 500
start_tokens = tf.ones([1], tf.int32) * gen_start_token_ph  # 生成時の batch_size は1

decoder = tf.contrib.seq2seq.BeamSearchDecoder(
    cell=cell,
    embedding=embeddings,
    start_tokens=start_tokens,  
    end_token=0,  # dummy
    initial_state=cell.zero_state(beam_width, tf.float32),
    beam_width=beam_width,
    output_layer=output_layer,
)

beam_decoder_output = tf.contrib.seq2seq.dynamic_decode(
    decoder=decoder,
    maximum_iterations=500,
    scope='generator_decode'
)[0]
generate_op = beam_decoder_output.predicted_ids

# Load and Convert Data

In [10]:
sentence_len = 50
batch_size = 512
data_path = 'data/natsume.txt'

In [11]:
class Tokenizer:
    def __init__(self, vocab):
        self.vocab = vocab
        self.rev_dict = {c: i for i, c in enumerate(vocab)}
        self.pad = 0
        self.bos = 1
        self.eos = 2
        self.unk = 3
    
    @classmethod
    def from_text(cls, text):
        char_freq_tuples = collections.Counter(text).most_common(vocab_size - 4)
        vocab, _ = zip(*char_freq_tuples)
        vocab = ['<pad>', '<bos>', '<eos>', '<unk>'] + list(vocab)
        return cls(vocab)

    @property
    def vocab_size(self):
        return len(self.vocab_size)
    
    def text2id(self, text):
        return [self.rev_dict[c] if c in self.rev_dict else self.unk for c in text]

    def id2text(self, ids):
        return ''.join(self.vocab[i] for i in ids)

In [12]:
with open(data_path) as f:
    text = f.read().replace('\n', '')

tokenizer = Tokenizer.from_text(text)
ids = tokenizer.text2id(text)

def split_ndlist(ndlist, size):
    return [np.array(ndlist[i - size:i]) for i in range(size, len(ndlist) + 1, size)]

# (1文字目, 2文字目), (2文字目, 3文字目), ... というペアを作る
# ある時刻の入力に対しその次時刻の出力を学習させるため
in_sequence_list = split_ndlist(ids[:-1], size=sentence_len)
out_sequence_list = split_ndlist(ids[1:], size=sentence_len)

in_batch_list = split_ndlist(in_sequence_list, batch_size)
out_batch_list = split_ndlist(out_sequence_list, batch_size)

# batch_size 個ごとに切り分け
batch_list = [
    {
        'in': in_batch,
        'out': out_batch,
        'len': np.array([len(seq) for seq in in_batch]),
    }
    for in_batch, out_batch
    in zip(in_batch_list, out_batch_list)
]

In [13]:
print(tokenizer.text2id('こんにちは😁'))
print(tokenizer.id2text([33, 27, 8, 51, 14, 3]))
print('batch list num: {}'.format(len(batch_list)))
print(batch_list[0])

[33, 27, 8, 51, 14, 3]
こんにちは<unk>
batch list num: 129
{'in': array([[  3,  77,   8, ...,  17, 224,  38],
       [ 12,  16,  55, ...,   4, 317,  14],
       [491,   3, 120, ...,  27,  25,  18],
       ...,
       [ 19,  25,  12, ..., 190, 255, 165],
       [ 11,  23,   4, ...,  10,  49, 266],
       [ 30,  12,  15, ...,   4,  14,  55]]), 'out': array([[ 77,   8,   3, ..., 224,  38,  12],
       [ 16,  55,  46, ..., 317,  14, 491],
       [  3, 120,   3, ...,  25,  18,   7],
       ...,
       [ 25,  12,  10, ..., 255, 165,  11],
       [ 23,   4,  19, ...,  49, 266,  30],
       [ 12,  15,  13, ...,  14,  55, 109]]), 'len': array([50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50,
       50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50,
       50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50,
       50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50,
       50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50

# Training

In [14]:
max_epoch = 50
save_path = 'tmp/rnnlm/model.ckpt'
log_dir = 'tmp/rnnlm/log/'
learning_rate = 0.001

In [15]:
if not os.path.isdir(os.path.dirname(save_path)):
    os.makedirs(os.path.dirname(save_path))
if not os.path.isdir(log_dir):
    os.makedirs(log_dir)

global_step = tf.Variable(0, name='global_step', trainable=False)
optimizer = tf.train.AdamOptimizer(learning_rate)
train_op = optimizer.minimize(loss_op, global_step=global_step)
tf.summary.scalar('loss', loss_op)
summary_op = tf.summary.merge_all()

In [ ]:
min_loss = 100000.0
sess = tf.Session()
summary_writer = tf.summary.FileWriter(log_dir, sess.graph)
saver = tf.train.Saver()

sess.run(tf.global_variables_initializer())
for epoch in range(max_epoch):
    random.shuffle(batch_list)
    for batch in tqdm(batch_list):
        feed_dict = {
            in_ph: batch['in'],
            out_ph: batch['out'],
            len_ph: batch['len'],
        }
        _, loss, summary, step = sess.run([train_op, loss_op, summary_op, global_step], feed_dict)
        summary_writer.add_summary(summary, step)
        if loss < min_loss:
            saver.save(sess, save_path)
            min_loss = loss
    print('epoch {}/{} - loss: {}'.format(epoch, max_epoch, loss))


# Restore

In [ ]:
load_path = 'learned_model/rnnlm/model.ckpt'
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.restore(sess, load_path)

# Generate

In [ ]:
start_char = '私'
generated_ids = sess.run(generate_op, {
    gen_start_token_ph:  tokenizer.text2id(start_char)[0]
})[0, :, 0]
generated_text = start_char + tokenizer.id2text(generated_ids)
print(generated_text)